# 1. Backend em Python
Raspa dados (*web-scrap*) para visualização de repositório da [Fundação Sistema Estadual de Análise de Dados Estatísticos](https://www.seade.gov.br/)

## 1.1. Instala dependências

In [1]:
pip install urllib3 && pip freeze | grep -e urllib3

urllib3==2.0.7


## 1.2. Realiza raspagem de dados

In [2]:
from bs4 import BeautifulSoup
import datetime
import json
import os
import pytz
import sys
import urllib3

urllib3.disable_warnings()

# Desativar a verificação do certificado SSL
urllib3_pool_manager = urllib3.PoolManager(cert_reqs='CERT_NONE')

base_url = "https://repositorio.seade.gov.br/"

supported_formats = ["csv", "CSV"]


def get_page_html_by_url(target_url):
  http_response = urllib3_pool_manager.request("GET", target_url)

  return BeautifulSoup(http_response.data, 'html.parser')

def get_products_dict_from_products_page(target_html):
  category_items = target_html.find_all('li', class_='media-item')

  result = {}
  for item in category_items:
      title = item.find('h2', class_='media-heading').text.strip()
      link = item.find('a', class_='media-view')['href']
      result[title] = link

  return result

def get_product_page_main_title(target_page):
  parent_element = target_page.find('form', id='group-datasets-search-form')
  target_element = parent_element.find_next('h1')

  return f"{target_element.text.strip()}"


def get_dataset_list_from_datasets_page(target_html):
  dataset_ul_element = target_html.find("ul", class_="dataset-list")
  dataset_li_elements = dataset_ul_element.find_all("li", class_="dataset-item")

  datasets = []
  for dataset_li_element in dataset_li_elements:
    li_div_element = dataset_li_element.find_next("div", class_="dataset-content")
    url = li_div_element.a["href"]
    name = li_div_element.a.text
    description = li_div_element.find_next("div").text
    resources = dataset_li_element.find_next('ul', class_="dataset-resources")

    formats = []
    if resources is not None:
        for format in resources.find_all('a'):
          formats.append(format['data-format'])
    else:
        continue

    dataset = {
          'url': url,
          'name': name,
          'description': description,
          'formats': formats
      }
    datasets.append(dataset)

  return datasets


def filter_dataset_list_by_format(dataset_list):
  result = []
  for dataset_item in dataset_list:
    for format in dataset_item['formats']:
      if format in supported_formats:
        result.append(dataset_item)

  return result


def get_dataset_resources_detail_from_dataset_page(target_html):
  resource_ul_element = target_html.find("ul", class_="resource-list")
  resource_li_elements = resource_ul_element.find_all("li", class_="resource-item")

  resource_files_list = []
  for resource_li_element in resource_li_elements:
    try:
      resource_name = resource_li_element.a["title"]
      resource_format = resource_li_element.a.find("span", class_="format-label").text
      resource_description = resource_li_element.p.text.strip()
      resource_url = resource_li_element.div.ul.find("a", class_="resource-url-analytics")["href"]

      if resource_format not in supported_formats:
        continue

      last_dot_index = resource_url.rfind('.')
      if resource_url[last_dot_index + 1:] not in supported_formats:
        continue

      try:
        response = urllib3_pool_manager.request("GET", resource_url)
        is_url_available = response.status == 200
      except Exception as e:
        print(f"ERROR3 falha ao acessar recurso:: {resource_url}")
        print(f"\nContinuando...")
        is_url_available = False

      if is_url_available != True:
        continue

      resource_file_info = {
        "resource_title": resource_name,
        "format": resource_format,
        "download_url": resource_url,
        "desctiption": resource_description
      }
      resource_files_list.append(resource_file_info)
    except Exception as e:
      print(f"\nERROR4 falha ao indexar:: {resource_name}, {resource_format}, {resource_url}")
      print(f"\nContinuando...")

  return resource_files_list


def get_dataset_resources_detail_list_from_dataset_list(dataset_filtered_list):
  result = []
  for dataset_item in dataset_filtered_list:
    try:
      target_html = get_page_html_by_url(f"{base_url}{dataset_item['url']}")
      dataset_resources_detail = get_dataset_resources_detail_from_dataset_page(target_html)

      if len(dataset_resources_detail) > 0:
        dataset = {
            "name": dataset_item["name"],
            "entity": "",
            "description": dataset_item["description"],
            "resources": dataset_resources_detail,
            "last_update": ""
        }
        result.append(dataset)

    except Exception as e:
      print(f"ERROR2: {base_url}{product_item[1]}")
      continue

  return result


def generate_json_file(products_dict):
  products = []

  datetime_now = datetime.datetime.now(pytz.timezone('Etc/GMT+3'))
  print(f"Raspagem de dados p/ pré-processamento iniciado em: {datetime_now}")
  print(f"\n\nEstimativa de finalização: {datetime_now + datetime.timedelta(minutes=15)}\n\n")
  total_items = len(products_dict.items())
  for idx, product_item in enumerate(products_dict.items()):

    # Calcula o percentual de conclusão
    progress_percent = (idx + 1) / total_items * 100
    sys.stdout.write(f"\rCarregando... {progress_percent:.2f}%")
    sys.stdout.flush()

    dataset_list = []
    try:
      product_page_html = get_page_html_by_url(f"{base_url}{product_item[1]}")
      product_page_main_title = get_product_page_main_title(product_page_html)
      dataset_complete_list = get_dataset_list_from_datasets_page(product_page_html)
      dataset_filtered_list = filter_dataset_list_by_format(dataset_complete_list)

      if len(dataset_filtered_list) < 1:
        continue
    except Exception as e:
      print(f"ERROR1: {base_url}{product_item[1]}")
      continue

    dataset_list = get_dataset_resources_detail_list_from_dataset_list(dataset_filtered_list)
    content_category = product_item[0]
    content_title = product_page_main_title

    if len(dataset_list) < 1:
        continue

    category = {
        "category": content_category,
        "title": content_title,
        "detail": f"{len(dataset_list)} compatível(is) com web-scrap (formato .csv)",
        "datasets": dataset_list
    }

    products.append(category)

    result = {
        "categories": products
    }

  json_file_path = "seade-repositorio.json"
  if os.path.exists(json_file_path):
      os.remove(json_file_path)

  with open(json_file_path, "w", encoding="utf-8") as outfile:
        json.dump(result, outfile, indent=4, ensure_ascii=False)

  return json_file_path

###

products_page_html = get_page_html_by_url(f"{base_url}{'/group'}")
products_page_html

products_dict = get_products_dict_from_products_page(products_page_html)
products_dict

product_page_html = get_page_html_by_url(f"{base_url}{products_dict['Seade Municípios']}")
product_page_html

product_page_main_title = get_product_page_main_title(product_page_html)
product_page_main_title

products_dict = get_products_dict_from_products_page(products_page_html)
print(f"Concluído com sucesso! {generate_json_file(products_dict)}")

Raspagem de dados p/ pré-processamento iniciado em: 2024-04-19 10:09:26.105842-03:00


Estimativa de finalização: 2024-04-19 10:24:26.105842-03:00


Carregando... 66.67%
ERROR4 falha ao indexar:: Dicionário de dados, CSV, https://repositorio.seade.gov.br/dataset/0cbb90f4-7ff8-4824-9e42-2bfb4738458c/resource/186b9c52-44ea-4b49-85f4-809ab3dcbddb/download/educacao_ideb_mun.csv

Continuando...
Carregando... 100.00%
ERROR4 falha ao indexar:: Página do produto no site do Seade, HTML, https://repositorio.seade.gov.br/dataset/6427da21-0c71-4a30-9042-d5608dd6b13f/resource/9f1a730d-ba19-4d2e-a47c-c532dcdf614e/download/leia-me_pesquisa_trajetorias_ocupacionais.pdf

Continuando...
Concluído com sucesso! seade-repositorio.json


# 2. Frontend em R
Utiliza dados raspados anteriormente para facilitar visualização e início da etapa de pré-processamento

## 2.1. Instala shiny-server e localtunnel na vm

In [8]:
!sudo apt-get install r-base
!sudo su - -c "R -e \"install.packages(c('shiny', 'ggExtra', 'rlist'), repos='https://cran.rstudio.com/')\""
!sudo apt-get install gdebi-core
!wget https://download3.rstudio.org/ubuntu-18.04/x86_64/shiny-server-1.5.21.1012-amd64.deb
!sudo gdebi -n shiny-server-1.5.21.1012-amd64.deb
!npm install localtunnel

checking for xmlHashSize in -lxml2... yes
Using built-in xmlHashSize
Checking DTD parsing (presence of externalSubset)...
checking for xmlHashSize in -lxml2... (cached) yes
Found xmlHashSize
checking for xmlOutputBufferCreateBuffer in -lxml2... yes
have xmlOutputBufferCreateBuffer()
checking for xmlDocDumpFormatMemoryEnc in -lxml2... yes
checking for libxml/xmlversion.h... yes
Expat:  FALSE
Checking for return type of xmlHashScan element routine.
No return value for xmlHashScan
xmlNs has a context field
Checking for cetype_t enumeration
No cetype_t enumeration defined in R headers.
checking for xmlsec1-config... no
nodegc default 
xml-debug default 
Version has XML_WITH_ZLIB
Version has xmlHasFeature()

****************************************
Configuration information:

Libxml settings

libxml include directory: -I/usr/include/libxml2
libxml library directory: -lxml2 -lz  -lxml2
libxml 2:                 -DLIBXML2=1

Compilation flags:         -DLIBXML -I/usr/include/libxml2 -DUSE_EXT

## 2.2. Cria servidor R Shiny (app.r)

In [ ]:
%%writefile app.r
library(shiny)
library(bslib)
library(dplyr)
library(ggplot2)
library(ggExtra)

penguins_csv <- "https://raw.githubusercontent.com/jcheng5/simplepenguins.R/main/penguins.csv"

df <- readr::read_csv(penguins_csv)
# Find subset of columns that are suitable for scatter plot
df_num <- df |> select(where(is.numeric), -Year)

ui <- page_sidebar(
  sidebar = sidebar(
    varSelectInput("xvar", "X variable", df_num, selected = "Bill Length (mm)"),
    varSelectInput("yvar", "Y variable", df_num, selected = "Bill Depth (mm)"),
    checkboxGroupInput(
      "species", "Filter by species",
      choices = unique(df$Species),
      selected = unique(df$Species)
    ),
    hr(), # Add a horizontal rule
    checkboxInput("by_species", "Show species", TRUE),
    checkboxInput("show_margins", "Show marginal plots", TRUE),
    checkboxInput("smooth", "Add smoother"),
  ),
  plotOutput("scatter")
)

server <- function(input, output, session) {
  subsetted <- reactive({
    req(input$species)
    df |> filter(Species %in% input$species)
  })

  output$scatter <- renderPlot({
    p <- ggplot(subsetted(), aes(!!input$xvar, !!input$yvar)) + list(
      theme(legend.position = "bottom"),
      if (input$by_species) aes(color = Species),
      geom_point(),
      if (input$smooth) geom_smooth()
    )

    if (input$show_margins) {
      margin_type <- if (input$by_species) "density" else "histogram"
      p <- ggExtra::ggMarginal(p, type = margin_type, margins = "both",
        size = 8, groupColour = input$by_species, groupFill = input$by_species)
    }

    p
  }, res = 100)
}

shinyApp(ui, server, options = list(host = "127.0.0.1", port = 8501))

Writing app.r


## 2.3. Inicia servidor R Shiny

In [ ]:
!echo -n "Password/Enpoint IP for localtunnel is: " && curl ipv4.icanhazip.com

Password/Enpoint IP for localtunnel is: 34.125.234.98


In [ ]:
!echo -n "Password/Enpoint IP for localtunnel is: " && curl ifconfig.me

In [ ]:
!npx localtunnel --port 8501 & Rscript app.r

## Json front ....

### Quick test

In [4]:
# activate R magic
%load_ext rpy2.ipython

In [ ]:
#%%R
#install.packages("rlist")

In [ ]:
%%R
library(jsonlite) # JSON parsing
library(rlist)

# Le json
#json_data <- jsonlite::read_json("/content/seade-repositorio.json")
json_data <- jsonlite::read_json("/content/bruno.json")

# Transforma json em objeto
categories <- list()
input_categories <- list()

#print(length(categories))
#rint(length(input_categories))
for(item in json_data$categories) {
    cat <- list(
      category = item$category,
      title = item$title,
      detail = item$detail,
      datasets = item$datasets
    )

    categories <- list.append(categories, cat)
    input_categories <- list.append(input_categories, cat$category)

    break
}
#print(categories)

target_category = ""
for(item in categories) {

    if ("Seade Indústria" == item$category)
      target_category <- item

    #print(target_category)
    break
}

datasets <- list()
input_datasets <- list()
target_dataset <- ""
#print(length(datasets))
#print(length(input_datasets))
for (dataset_item in target_category$datasets) {

    ds <- list(
      name = dataset_item$name,
      entity = dataset_item$entity,
      description = dataset_item$description,
      resources = dataset_item$resources
    )

    datasets <- list.append(datasets, ds)
    input_datasets <- list.append(input_datasets, ds$name)
    target_dataset <- ds

    break

}
#print(length(datasets))
#print(target_dataset$resources)
#print(length(input_datasets))


resources <- list()
input_resources <- list()
target_resource <- ""
#print(length(resources))
#print(length(input_resources))
#print(target_resource)
for (resource_item in target_dataset$resources) {

    rs <- list(
      title = resource_item$resource_title,
      format = resource_item$format,
      url = resource_item$download_url,
      desctiption = resource_item$desctiption
    )

    resources <- list.append(resources, rs)
    input_resources <- list.append(input_resources, resources$resource_title)
    target_resource <- rs

    break

}
#print(length(resources))
#print(length(input_resources))
print(target_resource)


### Final

In [10]:
!echo -n "Password/Enpoint IP for localtunnel is: " && curl ifconfig.me

Password/Enpoint IP for localtunnel is: 34.106.241.165

In [ ]:
!npx localtunnel --port 8501 & Rscript app_json.r


Attaching package: ‘shiny’

The following object is masked from ‘package:jsonlite’:

    validate

[..................] | fetchMetadata: sill resolveWithNewModule localtunnel@2.0
Listening on http://127.0.0.1:8501
npx: installed 22 in 2.338s
your url is: https://rude-ads-wink.loca.lt
  99: renderUI
  98: func
  85: renderFunc
  84: output$uiOutputRecurso
   3: runApp
   2: print.shiny.appobj
   1: <Anonymous>
  102: renderPrint
  101: func
   85: renderFunc
   84: output$object_details
    3: runApp
    2: print.shiny.appobj
    1: <Anonymous>
  102: renderPrint
  101: func
   85: renderFunc
   84: output$object_details
    3: runApp
    2: print.shiny.appobj
    1: <Anonymous>


In [100]:
%%writefile app_json.r
library(jsonlite)
library(rlist)
library(shiny)

# Carrega arquivo JSON
json_data <- jsonlite::read_json("/content/seade-repositorio.json")

# Carrega categories & input_categories para exebir na tela
categories <- list()
input_categories <- list()
for(category_item in json_data$categories) {
    cat <- list(
      category = category_item$category,
      title = category_item$title,
      detail = category_item$detail,
      datasets = category_item$datasets
    )

    categories <- list.append(categories, cat)
    input_categories <- list.append(input_categories, cat$category)
}

ui <- fluidPage(
  titlePanel("Repositório SEADE"),
  sidebarLayout(
    sidebarPanel(
      selectInput("inputProduto", "Selecione o Produto", choices = input_categories),
      uiOutput("uiOutputConjunto"),
      uiOutput("uiOutputRecurso")
    ),
    mainPanel(
      h3("Detalhes do Recurso"),
      verbatimTextOutput("object_details")
    )
  )
)

server <- function(input, output, session) {

  output$uiOutputConjunto <- renderUI({

    # Identifica produto selecionado
    target_category <- list()
    for(item in categories) {
      if (input$inputProduto == item$category) {
        target_category <- list(
          category = item$category,
          title = item$title,
          detail = item$detail,
          datasets = item$datasets
        )
        break
      }
    }

    # Carrega input_datasets para exibir na tela
    input_datasets <- list()
    for (dataset_item in target_category$datasets) {
        input_datasets <- list.append(input_datasets, dataset_item$name)
    }

    selectInput(
      "inputConjunto",
      "Selecione o Conjunto de Dados",
      choices = input_datasets
      )

  })

  output$uiOutputRecurso <- renderUI({

    # Identifica produto selecionado
    target_category <- list()
    for(item in categories) {
      if (input$inputProduto == item$category) {
        target_category <- list(
          category = item$category,
          title = item$title,
          detail = item$detail,
          datasets = item$datasets
        )
        break
      }
    }

    #Identifica dataset selecionado
    input_resources <- list()
    for (dataset_item in target_category$datasets) {
      if (input$inputConjunto == dataset_item$name) {
        for (resource_item in dataset_item$resources) {
          input_resources <- list.append(input_resources, resource_item$resource_title)
        }
        break
      }
    }

    selectInput(
      "inputRecurso",
      "Selecione o Recurso",
      choices = input_resources
      )
  })

  output$object_details <- renderPrint({
    # Identifica produto, dataset e recurso selecionado
    target_resource <- list()
    for(category_item in categories) {
      if (input$inputProduto == category_item$category) {
        for (dataset_item in category_item$datasets) {
          if (input$inputConjunto == dataset_item$name) {
            for (resource_item in dataset_item$resources) {
              if (input$inputRecurso == resource_item$resource_title) {
                target_resource <- list(
                  title = resource_item$resource_title,
                  format = resource_item$format,
                  url = resource_item$download_url,
                  description = resource_item$desctiption
                )
              }
              break
            }
            break
          }
        }
        break
      }
    }

    list(
      "Título" = target_resource$title,
      "Descrição" = target_resource$format,
      "Formato" = target_resource$url,
      "URL" = target_resource$description
    )
  })
}

shinyApp(ui, server, options = list(host = "127.0.0.1", port = 8501))

Overwriting app_json.r
